### This is first load on dataframe "jpx-tokyo-stock-exchange-prediction/stock_list.csv'"
##### Overall dataset of unique stocks in the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pymc as pm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../../jpx-tokyo-stock-exchange-prediction/stock_list.csv')
df.rename(columns={'Section/Products': 'Section', '33SectorName':'Sector_one', '17SectorName':'Sector_two'}, inplace=True)
df['TradeDate'] = pd.to_datetime(df['TradeDate'], unit='s')

In [ ]:
print(f'We have {len(df.SecuritiesCode.unique())} unique stocks in our dataset all with metadata in 15 additional columns : {df.shape[0]} x {df.shape[1]}.')
print(f'We have {len(df.Section.unique())} unique sections in our dataset.')
print(f'We have {len(df.NewMarketSegment.unique())} unique market segments in our dataset.')
print(f'We have {len(df.Sector_one.unique())} unique 33 Sector Names in our dataset.Some are None values, for example : {list(df.Sector_one.unique())[0:5]}')
print(f'We have {len(df.Sector_two.unique())} unique 17 Sector Names in our dataset.Some are None values, for example : {list(df.Sector_two.unique())[0:5]}')

### Loading train_files/stock_prices.csv that is the main stock trading data

In [ ]:
df_sp = pd.read_csv('../../jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
df_sp['Date'] = pd.to_datetime(df_sp['Date'])

In [ ]:
df_sp.head()

## Investigate a single stock to see how PACF works

In [ ]:
security = df_sp[df_sp["SecuritiesCode"] == 1376]
print(f'Security {1376} with number of closing prices : {security.shape[0]} and missing values : {security.Close.isnull().sum()}')
security['Close'].fillna(security['Close'].mean(), inplace=True)
security['Close'].plot(figsize=(16,4),legend=True)

# pcaf
correlations,conf_intervals = sm.tsa.stattools.pacf(security['Close'], nlags=20, method='ywm', alpha=0.05)
correl = correlations.tolist()[1:]
# First lag, lag 0 is always t today so it is 1
sm.graphics.tsa.plot_pacf(security['Close'], method="ywm",lags=20,alpha=0.05)
# sm.graphics.tsa.plot_acf(security['Close'],lags=100)
plt.show()


In [ ]:
correl

## Bootstrap betas from first 1000 stocks

In [ ]:
# calculate pcaf for first 100 stocks with 20 lags
correlation_matrix = np.zeros((len(df_sp.SecuritiesCode.unique()[0:1000]), 20))
for i, security in enumerate(list(df_sp.SecuritiesCode.unique())[0:1000]):
    stock = df_sp[df_sp["SecuritiesCode"] == security]
    stock['Close'].fillna(stock['Close'].mean(), inplace=True)
    correlations, conf_intervals = sm.tsa.stattools.pacf(stock['Close'], nlags=20, method='ywm', alpha=0.05)
    correlation_matrix[i,:] = correlations.tolist()[1:]
    

In [ ]:
# Bootstrap confidence intervals for each betas:
from scipy.stats import bootstrap
import seaborn as sns
rng = np.random.default_rng()

data = (correlation_matrix[:,1],)  # samples must be in a sequence
# Bootstrap for mean
res_mean = bootstrap(data, np.mean, confidence_level=0.95, random_state=rng)

# Bootstrap for standard deviation
res_std = bootstrap(data, np.std, confidence_level=0.95, random_state=rng)

# Creating subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Histogram and KDE for bootstrap mean
sns.histplot(res_mean.bootstrap_distribution, bins=25, kde=True, color='blue', alpha=0.7, ax=axs[0])
axs[0].set_title('Bootstrap Distribution of Mean')
axs[0].set_xlabel('Mean Value')
axs[0].set_ylabel('Frequency')

# Histogram and KDE for bootstrap standard deviation
sns.histplot(res_std.bootstrap_distribution, bins=25, kde=True, color='green', alpha=0.7, ax=axs[1])
axs[1].set_title('Bootstrap Distribution of Standard Deviation')
axs[1].set_xlabel('Standard Deviation Value')
axs[1].set_ylabel('Frequency')


# Display the plots
plt.tight_layout()
plt.show()


## Create baseline bayesian model

In [ ]:
## Load data for one stock:
security = df_sp[df_sp["SecuritiesCode"] == 1376]
print(f'Security {1376} with number of closing prices : {security.shape[0]} and missing values : {security.Close.isnull().sum()}')
security['Close'].fillna(security['Close'].mean(), inplace=True)

#### This is the idea of lags with np.empty_like...
So now if these were my prices:
my_prices_example = [1,2,3,4,5] \
beta_t_minus_1 = [0,0,0,0,0]\
beta_t_minus_1[0] = np.nan -> [nan,0,0,0,0]\
beta_t_minus_1[1:] = my_prices_examples -> [nan, 1,2,3,4]\
Meaning i am trying to predict 5, given the 4 lags: 4 ,3 ,2, 1

In [ ]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

# Your stock price data
cp_1376 = security["Close"].to_numpy()

# Split the data into train and test sets (80% train, 20% test)
split_index = int(len(cp_1376) * 0.8)
train = cp_1376[:split_index]
test = cp_1376[split_index:]

# Function to create lagged features
def create_lagged_features(data, n_lags=5):
    lagged_data = np.zeros((len(data) - n_lags, n_lags))
    for i in range(n_lags, len(data)):
        lagged_data[i - n_lags] = data[i - n_lags:i]
    return lagged_data

# Create lagged features for train and test
X_train = create_lagged_features(train)
y_train = train[5:]
X_test = create_lagged_features(test)
y_test = test[5:]

# Define the Bayesian model
with pm.Model() as model:
    # Priors for unknown model parameters
    beta = pm.Normal("beta", mu=0, sigma=1, shape=5) # 5, as we have 5 lags
    sigma = pm.HalfNormal("sigma", sigma=1)

    # Likelihood (sampling distribution) of observations
    y_obs = pm.Normal("y_obs", mu=pm.math.dot(X_train, beta), sigma=sigma, observed=y_train)

    # Sample from the posterior
    trace = pm.sample(1000, return_inferencedata=True)

# Analyze the results
pm.plot_trace(trace)

# Predictions on test set
beta_samples = trace.posterior["beta"].values.reshape(-1, 5)
predictions = np.dot(X_test, beta_samples.T)  # Using matrix multiplication for predictions

# Calculate the mean and credible interval for the predictions
pred_mean = np.mean(predictions, axis=1)
pred_hpd = np.percentile(predictions, [5, 95], axis=1)  # Ensure this is calculated for each prediction

#pred_hpd = np.array(), np.array()
#pred_hpd = lower 95% confidence interval, upper 95% confidence interval

# Calculate the error margins
lower_error = pred_mean - pred_hpd[0, :]
upper_error = pred_hpd[1, :] - pred_mean

# Assuming y_test is the correct length, adjust the range accordingly
x_range = range(len(y_test))

plt.figure(figsize=(10, 6))
plt.plot(x_range, y_test, label='Actual Data', color='blue')  # Plot the actual data
plt.plot(x_range, pred_mean, label='Predicted Mean', color='red')  # Plot the mean prediction

# Plot the error margins as filled areas
plt.fill_between(x_range, pred_mean - lower_error, pred_mean + upper_error, color='red', alpha=0.9, label='95% Credible Interval')

plt.xlabel('Time (Test Set)')
plt.ylabel('Stock Price')
plt.title('Actual vs Predicted Stock Prices with Error Margins (Test Set)')
plt.legend()
plt.show()


